# Retrieve Catalog Data from `catalogue.json`

The file `catalogue.json` contains all the catalog data for an [ORACC](http://oracc.org) project (for general information, see the [Oracc Open Data](http://oracc.org/doc/opendata) page). The `zip` that contains all JSON files of a particular project can be found at `http://build-oracc.museum.upenn.edu/json/[PROJECT].zip`. In the URL replace [PROJECT] with your project or sub-project name (e.g. `dcclt` or `cams/gkab`).

The main node in a `catalogue.json` file is called `members`. This node contains the information of all the fields and all the entries in the project catalog.

In [1]:
import pandas as pd
import zipfile
import json
import requests
import errno
import os
import tqdm

## 0 Create Directories, if Necessary
The two directories needed for this script are `jsonzip` and `output`. If they do not exist they are created, else: do nothing.

For the code, see [Stack Overflow](http://stackoverflow.com/questions/18973418/os-mkdirpath-returns-oserror-when-directory-does-not-exist).

In [2]:
directories = ['jsonzip', 'output']
for d in directories:
    try:
        os.mkdir(d)
    except OSError as exc:
        if exc.errno !=errno.EEXIST:
            raise
        pass

## 1.1 Input Project Name
We will download and manipulate one catalog file at a time. Different [ORACC](http://oracc.org) projects use different fields in their catalogs, the catalogs are not mutually compatible.

Provide a project name. Note that subprojects must be processed separately, they are not included in the main project. A subproject is named `[PROJECT]/[SUBPROJECT]`, for instance `saao/saa01`.

In [10]:
projects = input('Project(s): ').lower().strip() # lowercase user input and strip accidental spaces

Project(s): saao/saa11


## 1.2 Split the List of Projects
Split the list of projects and create a list of project names.

In [11]:
p = projects.split(',')               # split at each comma and make a list called `p`
p = [x.strip() for x in p]        # strip spaces left and right of each entry in `p`

## 1.2 Download the ZIP files
Download all the `json` files from `http://build-oracc.museum.upenn.edu/json/`. The file is called `[PROJECT].zip` (for instance: `dcclt.zip`). For subprojects the file is called `[PROJECT-SUBPROJECT].zip` (for instance `cams-gkab.zip`). 

For larger projects (such as [DCCLT](http://oracc.org/dcclt)) the `zip` file may be 25Mb or more. Downloading may take some time and it may be necessary to chunk the downloading process. The `iter_content()` function in the `requests` library takes care of that. For the chunking code see [this page](https://www.smallsurething.com/how-to-read-a-file-properly-in-python/).

If you have downloaded the files by hand (and put them in the `jsonzip` directory) you may skip this cell and jump directly to section ...

In [12]:
non_existent = []
CHUNK = 16 * 1024
for project in tqdm.tqdm(p):
    project = project.replace('/', '-')
    url = "http://build-oracc.museum.upenn.edu/json/" + project + ".zip"
    file = 'jsonzip/' + project + '.zip'
    r = requests.get(url)
    if r.status_code == 200:
        print("Downloading " + url + " saving as " + file)
        with open(file, 'wb') as f:
            for c in r.iter_content(chunk_size=CHUNK):
                f.write(c)
    else:
        print(url + " does not exist.")
        non_existent.append(project)
p = [i for i in p if i not in non_existent] # remove non-existing project names from list

  0%|                                                    | 0/1 [00:00<?, ?it/s]

100%|████████████████████████████████████████████| 1/1 [00:02<00:00,  2.06s/it]


## 2 Extract Catalogue Data from `JSON` files
The code in this cell will iterate through the list of projects entered above (1.1). For each project the `JSON` zip file, named `[PROJECT].zip` is located in the directory `jsonzip`. Each of these `zip` files includes a file called `catalogue.json`. This file is extracted and read with the command `json.loads()`, which reads the json data and transforms it into a JSON object - a sequence of names and values.

The JSON object is transformed into a Pandas Dataframe. The dataframe needs to be transposed (`.T`), so that the P, Q, and X numbers become indexes or row names (rather than column names), and each column represents a field in the catalog.  The individual dataframes (one for each project requested) are concatenated. Since individual [ORACC](http://oracc.org) project catalogs may have different fields, the dataframes may have different column names. By default Pandas concatenation uses an `outer join` so that all column names of all the catalogs are preserved.

In [13]:
df = pd.DataFrame() # create an empty dataframe
for project in p:
    file = "jsonzip/" + project.replace("/", "-") + ".zip"
    try:
        z = zipfile.ZipFile(file)       # create a Zipfile object
    except:
        print(file + " does not exist or is not a proper ZIP file")
        continue
    try:
        st = z.read(project + '/catalogue.json').decode('utf-8')  #read and decode the catalogue.json file of one project
                                                                # the result is a string object
    except:
        print(project + '/catalogue.json' + ' is not available or not complete')
        continue
    cat = json.loads(st)
    cat = cat['members']  # select the 'members' node 
    for item in cat.values():
        item["project"] = project # add project name as separate field
    cat_df = pd.DataFrame(cat).T
    df = pd.concat([df, cat_df])

## Create Dataframe
 Creating a Dataframe is not necessary, one may also manipulate the dictionary directly, but for demonstration purposes the Dataframe is a handy format. In manipulating the dictionary directly it is important to keep in mind that not all catalog fields have data for all entries, which means that not all dictionary keys are available for each P, Q, or X number.

The function `fillna('')` will put a blank (instead of `NaN`) in all fields that have no entry.

Example code for slicing the dictionary to select all entries that have `provenience = 'Ur'`:
> `urcat = {key:value for key, value in d.items() if 'provenience' in d[key] and d[key]['provenience'] == 'Ur'}

In [14]:
df = df.fillna('')
df

,ch_name,ch_no,ch_num_name,credits,date,designation,display_name,genre,id_text,images,...,public,publication_history,script,script_remarks,script_type,short_title,title,trans,vol_title,volume
P237876,Schedules of Land and People,Ch. 10,Ch. 10 (Schedules of Land and People),Copyright F. Mario Fales and J. Nicholas Postg...,,SAA 11 227,SAA 11 227,administrative record,P237876,[],...,no,"SAA 11, 227",Neo-Babylonian,inscribed,Cuneiform,Administrative Records 2,Report on Sown Land,[en],"Imperial Administrative Records, Part II: Prov...",SAA 11
P313588,Deportees and Displaced Persons,Ch. 8,Ch. 8 (Deportees and Displaced Persons),Copyright F. Mario Fales and J. Nicholas Postg...,,SAA 11 145,SAA 11 145,administrative record,P313588,[],...,no,"SAA 11, 145",Neo-Assyrian,inscribed,Cuneiform,Administrative Records 2,Difficulty of Obtaining Justice? A Memorandum,[en],"Imperial Administrative Records, Part II: Prov...",SAA 11
P313651,Geographical Lists,Ch. 1,Ch. 1 (Geographical Lists),Copyright F. Mario Fales and J. Nicholas Postg...,,SAA 11 012,SAA 11 012,administrative record,P313651,[],...,no,"SAA 11, 012",Neo-Assyrian,inscribed,Cuneiform,Administrative Records 2,Note of Towns with Persons in Charge,[en],"Imperial Administrative Records, Part II: Prov...",SAA 11
P313736,Harran Census,Ch. 9,Ch. 9 (Harran Census),Copyright F. Mario Fales and J. Nicholas Postg...,,SAA 11 200,SAA 11 200,administrative record,P313736,[],...,no,"SAA 11, 200",Neo-Assyrian,inscribed,Cuneiform,Administrative Records 2,From an Inventory of Families,[en],"Imperial Administrative Records, Part II: Prov...",SAA 11
P313842,Building Texts,Ch. 2,Ch. 2 (Building Texts),Copyright F. Mario Fales and J. Nicholas Postg...,,SAA 11 019,SAA 11 019,administrative record,P313842,[],...,no,"SAA 11, 019",Neo-Assyrian,inscribed,Cuneiform,Administrative Records 2,Building Progress Report,[en],"Imperial Administrative Records, Part II: Prov...",SAA 11
P313928,Domestic Animals,Ch. 5,Ch. 5 (Domestic Animals),Copyright F. Mario Fales and J. Nicholas Postg...,,SAA 11 099,SAA 11 099,administrative record,P313928,[],...,no,"SAA 11, 099",Neo-Assyrian,inscribed,Cuneiform,Administrative Records 2,List of Commodities Including Animals,[en],"Imperial Administrative Records, Part II: Prov...",SAA 11
P314083,Miscellaneous Commodities,Ch. 3,Ch. 3 (Miscellaneous Commodities),Copyright F. Mario Fales and J. Nicholas Postg...,,SAA 11 047,SAA 11 047,administrative record,P314083,[],...,no,"SAA 11, 047",Neo-Assyrian,inscribed,Cuneiform,Administrative Records 2,Fragment of Offering List,[en],"Imperial Administrative Records, Part II: Prov...",SAA 11
P314131,Deportees and Displaced Persons,Ch. 8,Ch. 8 (Deportees and Displaced Persons),Copyright F. Mario Fales and J. Nicholas Postg...,,SAA 11 152,SAA 11 152,administrative record,P314131,[],...,no,"SAA 11, 152",Neo-Assyrian,inscribed,Cuneiform,Administrative Records 2,Fragment of a List with Female Temple Staff,[en],"Imperial Administrative Records, Part II: Prov...",SAA 11
P314132,Miscellaneous Commodities,Ch. 3,Ch. 3 (Miscellaneous Commodities),Copyright F. Mario Fales and J. Nicholas Postg...,,SAA 11 034,SAA 11 034,administrative record,P314132,[],...,no,"SAA 11, 034",Neo-Assyrian,inscribed,Cuneiform,Administrative Records 2,Tribute(?) Bowls from the West,[en],"Imperial Administrative Records, Part II: Prov...",SAA 11
P314197,Deportees and Displaced Persons,Ch. 8,Ch. 8 (Deportees and Displaced Persons),Copyright F. Mario Fales and J. Nicholas Postg...,,SAA 11 168,SAA 11 168,administrative record,P314197,[],...,no,"SAA 11, 168",Neo-Assyrian,inscribed,Cuneiform,Administrative Records 2,Deportees(?) from the North and East,[en],"Imperial Administrative Records, Part II: Prov...",SAA 11


## Select Relevant Fields
[ORACC](http://oracc.org) catalogs may have custom fields, the only fields that are obligatory are `id_text` (the P, Q, or X number that identifies the text, for instance "P243546") and `designation` (the human-readable reference, for instance "VS 17, 012"). In order to find out which fields are available one may list the column names of the DataFrame.

First display all available fields, then select the ones that are relevant for the task at hand. 

In [15]:
df.columns

Index(['ch_name', 'ch_no', 'ch_num_name', 'credits', 'date', 'designation',
       'display_name', 'genre', 'id_text', 'images', 'last_mod_by',
       'last_modified', 'material', 'museum_no', 'object_type', 'period',
       'pleaides_id', 'pleiades_coord', 'pleiades_id', 'primary_publication',
       'project', 'provenience', 'public', 'publication_history', 'script',
       'script_remarks', 'script_type', 'short_title', 'title', 'trans',
       'vol_title', 'volume'],
      dtype='object')

In [16]:
df1 = df[['designation', 'period', 'provenience',
        'museum_no', 'project']]
df1

,designation,period,provenience,museum_no,project
P237876,SAA 11 227,Neo-Assyrian,Nineveh,K 00764,saao/saa11
P313588,SAA 11 145,Neo-Assyrian,Nineveh,K 01441,saao/saa11
P313651,SAA 11 012,Neo-Assyrian,Nineveh,K 05131,saao/saa11
P313736,SAA 11 200,Neo-Assyrian,Nineveh,K 07294,saao/saa11
P313842,SAA 11 019,Neo-Assyrian,Nineveh,K 11933+,saao/saa11
P313928,SAA 11 099,Neo-Assyrian,Nineveh,K 14244,saao/saa11
P314083,SAA 11 047,Neo-Assyrian,Nineveh,K 15601,saao/saa11
P314131,SAA 11 152,Neo-Assyrian,Nineveh,K 16036,saao/saa11
P314132,SAA 11 034,Neo-Assyrian,Nineveh,K 16037,saao/saa11
P314197,SAA 11 168,Neo-Assyrian,Nineveh,K 16517,saao/saa11


## Manipulate
The Dataframe may now be manipulated with standard Pandas methods. The example code selects the texts from Ur.
> `ur = df1[df1.provenience == "Ur"]`

## Save
Save the resulting data set as a `csv` file. `UTF-8` encoding is the encoding with the widest support in text analysis (and also the encoding used by [ORACC](http://oracc.org)). If you intend to use the catalog file in Excel, however, it is better to use `utf-16` encoding.

In [17]:
filename = 'catalog.csv'
with open('output/' + filename, 'w', encoding='utf-8') as w:
    df1.to_csv(w)